In [ ]:
import csv
import math
import re
from math import *
import numpy as np 
import pandas as pd 
import pickle

import category_encoders as ce

import xgboost as xgb
from xgboost import XGBRegressor
from catboost import CatBoostClassifier, CatBoostRegressor, Pool
import lightgbm as lgb

from sklearn.model_selection import GridSearchCV, train_test_split, KFold, StratifiedKFold

from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, MinMaxScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, MissingIndicator

from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression, BayesianRidge, RidgeClassifier, RidgeClassifierCV, TheilSenRegressor, HuberRegressor
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier, StackingClassifier 
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import GridSearchCV, train_test_split, KFold, StratifiedKFold

from sklearn.metrics import roc_auc_score, auc, accuracy_score, mean_squared_error

from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier, StackingClassifier 
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, BaggingRegressor, ExtraTreesRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor, StackingRegressor 
#from from sklearn.ensemble  VoltingClassifier
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, SGDRegressor, ElasticNet, ElasticNetCV, Lars
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor
from sklearn.gaussian_process import GaussianProcessRegressor

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [ ]:
#!ls /kaggle/input/ncaam-march-mania-2021

In [ ]:
FILE_TRAIN_2015 = "google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MEvents2015.csv"
FILE_TRAIN_2016 = "google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MEvents2016.csv"
FILE_TRAIN_2017 = "google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MEvents2017.csv"
FILE_TRAIN_2018 = "google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MEvents2018.csv"
FILE_TRAIN_2019 = "google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MEvents2019.csv"
FILE_TEST = "google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MSampleSubmissionStage1_2020.csv"

NAN_STRING_TO_REPLACE = 'zz'
NAN_VALUE_FLOAT = 8888.0
NAN_VALUE_INT = 8888
NAN_VALUE_STRING = '8888'

BATCH_SIZE = 1000
EPOCHS = 5
N_NEURONS = 10

SEED = 8888
SPLITS = 5

SMOOTHING = 0.2
OTHER_NAN = 0

IMPUTING_STRATEGY = 'median'

PARAMS_CATBOOST = dict()
# PARAMS_CATBOOST['logging_level'] = 'Silent'
PARAMS_CATBOOST['eval_metric'] = 'Logloss'
PARAMS_CATBOOST['custom_metric'] = 'Logloss'
PARAMS_CATBOOST['loss_function'] = 'Logloss'
PARAMS_CATBOOST['iterations'] = 125 # best 125
PARAMS_CATBOOST['od_type'] = 'Iter' # IncToDec, Iter
PARAMS_CATBOOST['random_seed'] = SEED
PARAMS_CATBOOST['learning_rate'] = 0.003 # alpha, default 0.03 if no l2_leaf_reg
PARAMS_CATBOOST['task_type'] = 'CPU'
PARAMS_CATBOOST['use_best_model']: True
PARAMS_CATBOOST['l2_leaf_reg'] = 3.0 # lambda, default 3, S: 300

PARAM_LGB = dict()
PARAM_LGB['metric'] = 'rmse'
PARAM_LGB['objective'] = 'regression'

'''
PARAMS_CATBOOST['depth'] = 3
PARAMS_CATBOOST['bagging_temperature'] = 0.8
PARAMS_CATBOOST['random_strength']: 0.8
PARAMS_CATBOOST['bootstrap_type'] = 'Bayesian'
PARAMS_CATBOOST['nan_mode'] = 'Min'
PARAMS_CATBOOST['thread_count'] = 4
PARAMS_CATBOOST['l2_leaf_reg'] = 300 # lambda, default 3
'''

PARAMS_CATBOOST_REGRESSOR = dict()
#PARAMS_CATBOOST_REGRESSOR['logging_level'] = 'Silent'
PARAMS_CATBOOST_REGRESSOR['eval_metric'] = 'RMSE'
PARAMS_CATBOOST_REGRESSOR['custom_metric'] = 'RMSE'
PARAMS_CATBOOST_REGRESSOR['loss_function'] = 'RMSE'
PARAMS_CATBOOST_REGRESSOR['iterations'] = 100
PARAMS_CATBOOST_REGRESSOR['od_type'] = 'Iter' # IncToDec, Iter
PARAMS_CATBOOST_REGRESSOR['random_seed'] = SEED
PARAMS_CATBOOST_REGRESSOR['learning_rate'] = 0.1 # alpha, default 0.03 if no l2_leaf_reg
PARAMS_CATBOOST_REGRESSOR['task_type'] = 'CPU'
PARAMS_CATBOOST_REGRESSOR['use_best_model']: True
PARAMS_CATBOOST_REGRESSOR['l2_leaf_reg'] = 50.0 # lambda, default 3, S: 300

'''
PARAMS_CATBOOST_REGRESSOR['depth'] = 3
PARAMS_CATBOOST_REGRESSOR['bagging_temperature'] = 0.8
PARAMS_CATBOOST_REGRESSOR['random_strength']: 0.8
PARAMS_CATBOOST_REGRESSOR['bootstrap_type'] = 'Bayesian'
PARAMS_CATBOOST_REGRESSOR['nan_mode'] = 'Min'
PARAMS_CATBOOST_REGRESSOR['thread_count'] = 4
PARAMS_CATBOOST_REGRESSOR['l2_leaf_reg'] = 300 # lambda, default 3
'''

PARAMS_XGB = dict()
PARAMS_XGB['objective']='binary:logistic'
PARAMS_XGB['eval_metric'] = 'mae'
PARAMS_XGB['booster'] = 'gbtree'
PARAMS_XGB['eta'] = 0.02
PARAMS_XGB['subsample'] = 0.35
PARAMS_XGB['colsample_bytree'] = 0.7
PARAMS_XGB['num_parallel_tree'] = 10
PARAMS_XGB['min_child_weight'] = 40
PARAMS_XGB['gamma'] = 10
PARAMS_XGB['max_depth'] = 3


W_FEATURES = [
    'WTeamID', 
    'WFGM', 
    'WFGA', 
    'WFGM3', 
    'WFGA3', 
    'WFTM', 
    'WFTA', 
    'WOR', 
    'WDR', 
    'WAst', 
    'WTO', 
    'WStl', 
    'WBlk', 
    'WPF', 
    'WScore', 
    'Final_WTeam', 
    #'Semi_Final_WTeam', 
    'WTeam_W_count', 
    'WScore_mean',
    'WScore_median', 
    'WScore_sum',
    'WTeam_Seed',
    'WTeam_PerCent',
    'Diff_WTeam',
    'WFGA_min', 
    #'WFGA_max', 
    'WFGA_mean',
    'WFGA_median'
    #'WAst_mean',
    #'WBlk_mean'
]

L_FEATURES = [
    'LTeamID', 
    'LFGM', 
    'LFGA', 
    'LFGM3', 
    'LFGA3', 
    'LFTM', 
    'LFTA', 
    'LOR', 
    'LDR', 
    'LAst', 
    'LTO', 
    'LStl', 
    'LBlk', 
    'LPF', 
    'LScore',
    'Final_LTeam', 
    #'Semi_Final_LTeam', 
    'LTeam_L_count', 
    'LScore_mean',  
    'LScore_median', 
    'LScore_sum',
    'LTeam_Seed',
    'LTeam_PerCent',
    'Diff_LTeam',
    'LFGA_min', 
    #'LFGA_max', 
    'LFGA_mean', 
    'LFGA_median'
    #'LAst_mean',
    #'LBlk_mean'
]

W_FEATURES_1 = [
    'Final_WTeam', 
    #'Semi_Final_WTeam', 
    'WTeam_W_count', 
    'WScore_mean',
    'WScore_median', 
    'WScore_sum',
    'WTeam_Seed',
    'WTeam_PerCent',
    'Diff_WTeam',
    'WFGA_min', 
    #'WFGA_max', 
    'WFGA_mean', 
    'WFGA_median'
    #'WAst_mean',
    #'WBlk_mean'
   
]
L_FEATURES_1 = [
    'Final_LTeam', 
    #'Semi_Final_LTeam', 
    'LTeam_L_count', 
    'LScore_mean',  
    'LScore_median', 
    'LScore_sum',
    'LTeam_Seed',
    'LTeam_PerCent',
    'Diff_LTeam',
    'LFGA_min', 
    #'LFGA_max', 
    'LFGA_mean', 
    'LFGA_median'
    #'LAst_mean',
    #'LBlk_mean'
]

lgb_params = {'objective': 'binary',
              'metric': 'binary_logloss',
              'boosting': 'gbdt',
              'num_leaves': 32,
              'feature_fraction': 0.6,
              'bagging_fraction': 0.6,
              'bagging_freq': 5,
              'learning_rate': 0.05
}

xgb_params = {'max_depth':63,
              #'objective':'binary:logistic',
              'objective':'reg:squarederror',
              'min_child_weight': 10,
              'learning_rate': 0.0001,
              'eta'      :0.3,
              'subsample':0.8,
              #'lambda '  :4,
              'eval_metric':'logloss',
              #'n_estimators':2000,
              #'colsample_bytree ':0.9,
              'colsample_bylevel':1
              }


In [ ]:

'''
Description: Read Data from CSV file into Pandas DataFrame
'''
def read_data(inFile, sep=','):
    df_op = pd.read_csv(filepath_or_buffer=inFile, low_memory=False, encoding='utf-8', sep=sep)
    return df_op

'''
Description: Write Pandas DataFrame into CSV file
'''
def write_data(df, outFile):
    f = open(outFile+'.csv', 'w')
    r = df.to_csv(index=False, path_or_buf=f)
    f.close()

def print_submission_into_file(y_pred, df_test_id, algo=""):
    l = []
    for myindex in range(y_pred.shape[0]):
        Y0 = y_pred[myindex]
        l.insert(myindex, Y0)
    
    df_pred = pd.DataFrame(pd.Series(l), columns=["Pred"])
    df_result = pd.concat([df_test_id, df_pred], axis=1, sort=False)
     
    f = open('submission'+algo+'.csv', 'w')
    r = df_result.to_csv(index=False, path_or_buf=f)
    f.close()

    return df_result

def print_submission(y_pred, df_test_id):
    l = []
    for myindex in range(y_pred.shape[0]):
        Y0 = y_pred[myindex]
        l.insert(myindex, Y0)
    
    df_pred = pd.DataFrame(pd.Series(l), columns=["Pred"])
    df_result = pd.concat([df_test_id, df_pred], axis=1, sort=False)
     
    f = open('submission.csv', 'w')
    r = df_result.to_csv(index=False, path_or_buf=f)
    f.close()

    return df_result


def vectorize_more_weight_out(x):
    if x>0.5:
        return 0.9
    #elif x<0.5 and x>0.3:
    #    return 0.3
    
    else:
        return x

def vectorize_str(x):
    return str(x)


def vectorize_float(x):
    return float(x)

def vectorize_exp(x):
    r = math.exp(x)
    return r

def vectorize_expm1(x):
    r = math.expm1(x)
    return r

def vectorize_log(x):
    r = math.log(x)
    return r

def vectorize_log1p(x):
    r = math.log1p(x)
    return r

def vectorize_pred_result_063945(x):
    if x < 0.2:
        return 0.2
    elif x > 0.7:
        return 0.6
    else:
        return x

def vectorize_pred_result(x):
    if x < 0.02:
        return 0.02
    elif x > 0.95:
        return 0.95
    else:
        return x

def vectorize_more_weight_out(x):
    if x>0.5:
        return 0.9
    #elif x<0.5 and x>0.3:
    #    return 0.3
    
    else:
        return x
        
def log_loss(y_01, y_p):
    n = y_01.shape[0]
    v = np.multiply(y_01, np.log(y_p)) + np.multiply((1-y_01), np.log(1-y_p))
    
    res = -(np.sum(v)/float(n)) 
    return res


def set_aggregation(row, se_agg, se_col, r_col, op_col):
    df_s = se_agg[se_agg[se_col] == row[r_col]]
    df = df_s[df_s['Season']==row['Season']].reset_index(drop=True)
    if df.shape[0] == 0:
        return 0
    else:
        return df.at[0, op_col]


def get_value_for_count(team, team_name, team_count):
    if team in team_count.index:
        return team_count.loc[team, 'Count']
    else:
        return 0

def set_WLoc(row):
    if row==1:
        return 2
    elif row==2:
        return 1
    else:
        return 0

def get_value_for_count(team, team_name, team_count):
    if team in team_count.index:
        return team_count.loc[team, 'Count']
    else:
        return 0
    
def concat_row(r):
    if r['WTeamID'] < r['LTeamID']:
        res = str(r['Season'])+"_"+str(r['WTeamID'])+"_"+str(r['LTeamID'])
    else:
        res = str(r['Season'])+"_"+str(r['LTeamID'])+"_"+str(r['WTeamID'])
    return res

def write_label(r):
    if r['WTeamID'] < r['LTeamID']:
        return 1
    else:
        return 0

# Delete leaked from train
def delete_leaked_from_df_train(df_train, df_test):
    # Delete leaked from train
    dft = df_train.loc[:, ['Season','WTeamID','LTeamID']]
    df_train['Concats'] = df_train.apply(concat_row, axis=1)
    df2 = df_test[df_test['ID'].isin(df_train['Concats'].unique())]

    df_train_duplicates = df_train[df_train['Concats'].isin(df_test['ID'].unique())]
    df_train_idx = df_train_duplicates.index.values
    
    df_train = df_train.drop(df_train_idx)
    df_train = df_train.drop('Concats', axis=1)
    
    return df_train

def get_labels_df_train(df_train, df_test):
    # Delete leaked from train
    df_train['Concats'] = df_train.apply(concat_row, axis=1)
    
    df_train_good = df_train[df_train['Concats'].isin(df_test['ID'].unique())]
    
    #df_train_good = df_train[~df_train['Concats'].isin(df_test['ID'].unique())]
    df_train_good['Label'] = df_train_good.apply(write_label, axis=1)
    write_data(df_train_good, 'labels_train')
    

def replace_seed(s):
    s = s.replace('W', '1')
    s = s.replace('X', '2')
    s = s.replace('Y', '3')
    s = s.replace('Z', '4')
    
    if re.search('(a|b)', s):
        s = s.replace('a', '1')
        s = s.replace('b', '2')
    else:
        s = s+'0'
    res = - int(s)
    return res

def replace_seed_2(s):
    rank = 0
    seed = s
    if re.search('W', s):
        rank = '1'
        seed = seed.replace('W', '')
    elif re.search('X', s):
        rank = '2'
        seed = seed.replace('X', '')
    elif re.search('Y', s):
        rank = '3'
        seed = seed.replace('Y', '')
    elif re.search('Z', s):
        rank = '4'
        seed = seed.replace('Z', '')
    
    if re.search('a', s):
        seed = seed.replace('a', '1')
    if re.search('b', s):    
        seed = seed.replace('b', '2')
    else:
        seed = seed+'0'
       
    final_seed = seed+rank
    res = - int(final_seed)
    return res

def replace_seed_only(s):
    s = s.replace('W', '')
    s = s.replace('X', '')
    s = s.replace('Y', '')
    s = s.replace('Z', '')
    
    if re.search('(a|b)', s):
        s = s.replace('a', '')
        s = s.replace('b', '')
    else:
        s = s+'0'
     
    return int(s)

def replace_rank_only(s):
    if re.search('(a|b)', s):
        s = re.sub('(a|b)', '', s)
    
    s = re.sub('(\d)+', '', s)
    
    s = s.replace('W', '1')
    s = s.replace('X', '2')
    s = s.replace('Y', '3')
    s = s.replace('Z', '4')
         
    return int(s)

def convert_categoricals_to_numbers(df):
    sorted_EventType = sorted(df.EventType.dropna().unique())
    mapping_EventType = dict(zip(sorted_EventType, range(len(sorted_EventType))))
    df['EventType'] = df.loc[df.EventType.notnull(), 'EventType'].map(mapping_EventType)

    df = df.reset_index(drop=True)
    
    sorted_EventSubType = sorted(df.EventSubType.dropna().unique())
    mapping_EventSubType = dict(zip(sorted_EventSubType, range(len(sorted_EventSubType))))
    df['EventSubType'] = df.loc[df.EventSubType.notnull(), 'EventSubType'].map(mapping_EventSubType)
    
    return df

# Convert categories to numeric
def convert_data_to_numeric_2(df, df_all, ar_train_transformed=None, enc=None):
    columns_for_ordinal_encoder = ['EventType', 'EventSubType']
    
    enc_new = enc
    
    # Fillna with ZZ which will be the last value in alphabetical order. 
    df_ordinal = df[columns_for_ordinal_encoder].fillna(NAN_STRING_TO_REPLACE).applymap(lambda x: str(x))
    df_ordinal_all = df_all[columns_for_ordinal_encoder].fillna(NAN_STRING_TO_REPLACE).applymap(lambda x: str(x))
    ar_train_transformed_new = ar_train_transformed
    if enc == None and ar_train_transformed == None:
        enc_new = OrdinalEncoder(dtype=np.int16)
        enc_new.fit(df_ordinal_all)
        
        
        ar_train_transformed_new = enc_new.transform(df_ordinal_all)
    
    ar_ordinal_transformed = enc_new.transform(df_ordinal)
    count_columns = 0
    for cn in columns_for_ordinal_encoder:
        this_col_all = ar_train_transformed_new[:,count_columns]
        mx = this_col_all.max() # Find the index of ZZ, last one always, in our case, as OrdinalEncoder encodes by alphabetic order
        
        this_col_train = ar_ordinal_transformed[:,count_columns]
        this_col_nan_train = np.where(this_col_train==mx, NAN_VALUE_INT, this_col_train)
    
        ar_ordinal_transformed[:,count_columns] = this_col_nan_train
        count_columns = count_columns+1

    df_ordinal_transformed = pd.DataFrame(ar_ordinal_transformed, columns=columns_for_ordinal_encoder)
    
    df.update(df_ordinal_transformed)      
    
    # Fill NaN with -1 first : 
    df = df.fillna(-1)
     
    # Optimize :
    for cn in df.columns:
        cn_size = df[cn].unique().size
        
        if cn_size <= np.iinfo(np.int8).max:
            df[cn] = df[cn].astype('int8')
        elif cn_size > np.iinfo(np.int8).max  and cn_size <= np.iinfo(np.int16).max:
            df[cn] = df[cn].astype('int16')
        elif cn_size > np.iinfo(np.int16).max  and cn_size <= np.iinfo(np.int32).max:
            df[cn] = df[cn].astype('int32')    
    
    df = df.applymap(lambda x: NAN_VALUE_INT if x == -1 else x)
    
    return (df, ar_train_transformed_new, enc_new)


In [ ]:

PATH = "/kaggle/input/ncaam-march-mania-2021/MDataFiles_Stage2/"

df_mncaa_tourney_detailed_results = read_data(PATH+"MNCAATourneyDetailedResults.csv") # OK
df_regular_season_detailed_results = read_data(PATH+"MRegularSeasonDetailedResults.csv") # OK
df_tourney_seeds = read_data(PATH+"MNCAATourneySeeds.csv")
df_test = read_data(PATH+"MSampleSubmissionStage2.csv")

df_train = df_mncaa_tourney_detailed_results
df_train = delete_leaked_from_df_train(df_train, df_test)

# df_train = df_train.append(df_regular_season_detailed_results, ignore_index=True)
# df_train = df_train.drop('WLoc', axis=1)

# Seeds
df_tourney_seeds['SeedID'] = df_tourney_seeds['Seed'].apply(replace_seed_only)

df_train_tcr = df_mncaa_tourney_detailed_results

mapping_WLoc = {'N':0, 'A':1, 'H':2}
df_train['WLoc'] = df_train.loc[df_train.WLoc.notnull(), 'WLoc'].map(mapping_WLoc)

# Features to parse
features = df_train.columns

df_train_features = df_train[features]
df_train_features = df_train_features.fillna(NAN_VALUE_INT)

# Create simple imputer
si_mf = SimpleImputer(missing_values=NAN_VALUE_INT, strategy=IMPUTING_STRATEGY)
ar_train = si_mf.fit_transform(df_train_features)
df_train = pd.DataFrame(ar_train, columns=features)

df_train_tcr = df_train.copy()
df_train_tcr = df_train_tcr.fillna(NAN_VALUE_INT)
features_tcr = ['Season', 'WTeamID', 'LTeamID', 'WScore', 'LScore', 'NumOT']

# FINAL
df_train_tcr_final = df_train_tcr[df_train_tcr['DayNum']==154]
ar_tcr_final_teams = df_train_tcr_final.loc[:,['WTeamID', 'LTeamID']].to_numpy()
ar_tcr_final_teams = np.unique(ar_tcr_final_teams)
df_tcr_final_teams = pd.DataFrame(ar_tcr_final_teams)

df_tcr_final_teams_2 = ar_tcr_final_teams.flatten()

ar_final_teams_count = np.array(np.unique(df_tcr_final_teams_2, return_counts=True)).T
df_final_teams_count = pd.DataFrame(ar_final_teams_count, columns=['TeamID','Count'])

# SEMI FINAL
df_train_semi_final = df_train_tcr[df_train_tcr['DayNum']==152]
ar_semi_final_teams = df_train_semi_final.loc[:,['WTeamID', 'LTeamID']].to_numpy()
ar_semi_final_teams = np.unique(ar_semi_final_teams)
df_semi_final_teams = pd.DataFrame(ar_semi_final_teams)
df_semi_final_teams_2 = ar_semi_final_teams.flatten()
ar_semi_final_teams_count = np.array(np.unique(df_semi_final_teams_2, return_counts=True)).T
df_semi_final_teams_count = pd.DataFrame(ar_semi_final_teams_count, columns=['TeamID','Count'])

# Aggregates
wt_se_agg = df_train_tcr.groupby(['Season', 'WTeamID']).agg({'WScore':['sum','mean','median', 'count']})
lt_se_agg = df_train_tcr.groupby(['Season', 'LTeamID']).agg({'WScore':['sum','mean','median', 'count']})
wt_se_mean = df_train_tcr.groupby(['WTeamID', 'Season']).mean()

wt_se_agg.columns = ['sum', 'mean', 'median', 'count']
wt_se_agg = wt_se_agg.reset_index()

lt_se_agg.columns = ['sum', 'mean', 'median', 'count']
lt_se_agg = lt_se_agg.reset_index()

# Sum and mean
wt_mean = df_train_tcr.groupby('WTeamID').mean()
wt_sum = df_train_tcr.groupby('WTeamID').sum()
wt_median = df_train_tcr.groupby('WTeamID').median()
lt_mean = df_train_tcr.groupby('LTeamID').mean()
lt_sum = df_train_tcr.groupby('LTeamID').sum()
lt_median = df_train_tcr.groupby('LTeamID').median()

# Nb wins, lose
wt_count = df_train_tcr.groupby('WTeamID').size().to_frame()
lt_count = df_train_tcr.groupby('LTeamID').size().to_frame()

wt_count.columns = ['Count']
lt_count.columns = ['Count']

# Min
wt_min = df_train_tcr.groupby('WTeamID').min()
lt_min = df_train_tcr.groupby('LTeamID').min()

# Min
wt_max = df_train_tcr.groupby('WTeamID').max()
lt_max = df_train_tcr.groupby('LTeamID').max()


df_train['WTeam_Seed'] = df_train.loc[:,['Season','WTeamID']].apply(lambda row: set_aggregation(row, df_tourney_seeds, 'TeamID', 'WTeamID', 'SeedID'), axis=1)
df_train['LTeam_Seed'] = df_train.loc[:,['Season','LTeamID']].apply(lambda row: set_aggregation(row, df_tourney_seeds, 'TeamID', 'LTeamID', 'SeedID'), axis=1)

features = df_train.columns
df_train_features = df_train.fillna(NAN_VALUE_INT)

# Create simple imputer
si_mf = SimpleImputer(missing_values=NAN_VALUE_INT, strategy=IMPUTING_STRATEGY)
si_mf.fit(df_train_features)
   
df_test_id = df_test["ID"]
df_test = df_test["ID"].apply(lambda x: pd.Series(x.split("_"))).astype('int16')
df_test.columns = ['Season', 'WTeamID', 'LTeamID']

df_test['WTeam_Seed'] = df_test.loc[:,['Season','WTeamID']].apply(lambda row: set_aggregation(row, df_tourney_seeds, 'TeamID', 'WTeamID', 'SeedID'), axis=1)
df_test['LTeam_Seed'] = df_test.loc[:,['Season','LTeamID']].apply(lambda row: set_aggregation(row, df_tourney_seeds, 'TeamID', 'LTeamID', 'SeedID'), axis=1)

df_test['DayNum'] = NAN_VALUE_INT
df_test['NumOT'] = df_train_tcr['NumOT'].max()
df_test['WLoc'] = 0

imputation = 3

w_features = ['WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'WScore']
l_features = ['LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF', 'LScore']

features = df_train.columns
if imputation == 0:
    for cn in features:
        if cn in ['Season', 'WTeamID', 'LTeamID', 'WLoc', 'DayNum', 'WTeam_Seed', 'LTeam_Seed']:
            continue
        df_test[cn] = NAN_VALUE_INT

    # Impute to df_test
    df_test = df_test.fillna(NAN_VALUE_INT)
    ar_test = si_mf.transform(df_test)
    df_test = pd.DataFrame(ar_test, columns=features).astype('float64')        
elif imputation == 1:
    df_test['DayNum'] = df_train['DayNum'].median()
    df_test['NumOT']  = df_train['NumOT'].median()
     
    agg_strategy = 'mean'
    
    for i in range(len(w_features)):
        cn_w = w_features[i]
        cn_l = l_features[i]
        
        wt_agg = df_train.groupby(['Season', 'WTeamID']).agg({cn_w:['sum', 'mean', 'median']})
        wt_agg.columns = ['sum', 'mean', 'median']
        wt_agg = wt_agg.reset_index()
        df_test[cn_w] = df_test.loc[:, ['Season', 'WTeamID']].apply(lambda row: set_aggregation(row, wt_agg, 'WTeamID', 'WTeamID', agg_strategy), axis=1)
        
        lt_agg = df_train.groupby(['Season', 'LTeamID']).agg({cn_l:['sum', 'mean', 'median']})
        lt_agg.columns = ['sum', 'mean', 'median']
        lt_agg = lt_agg.reset_index()
        df_test[cn_l] = df_test.loc[:,['Season','LTeamID']].apply(lambda row: set_aggregation(row, lt_agg, 'LTeamID', 'LTeamID',agg_strategy), axis=1)
       
    '''
    for cn in w_features:
        # df_test[cn] = df_test['WTeamID'].map(wt_sum[cn])
        wt_agg = df_train.groupby(['Season', 'WTeamID']).agg({cn:['sum', 'mean', 'median']})
        wt_agg.columns = ['sum', 'mean', 'median']
        wt_agg = wt_agg.reset_index()
        df_test[cn] = df_test.loc[:, ['Season', 'WTeamID']].apply(lambda row: set_aggregation(row, wt_agg, 'WTeamID', 'WTeamID', agg_strategy), axis=1)

    for cn in l_features:
        #df_test[cn] = df_test['LTeamID'].map(lt_sum[cn])
        lt_se_agg = df_train.groupby(['Season', 'LTeamID']).agg({cn:['sum', 'mean', 'median']})
        lt_se_agg.columns = ['sum', 'mean', 'median']
        lt_se_agg = lt_se_agg.reset_index()
        df_test[cn] = df_test.loc[:,['Season','LTeamID']].apply(lambda row: set_aggregation(row, lt_se_agg, 'LTeamID', 'LTeamID',agg_strategy), axis=1)
    '''    

elif imputation == 2: # mean
    df_test['DayNum'] = df_train['DayNum'].mean()
    df_test['NumOT']  = df_train['NumOT'].mean()
 
    for i in range(len(w_features)):
        cn_w = w_features[i]
        cn_l = l_features[i]

        df_test[cn_w] = df_test['WTeamID'].map(wt_mean[cn_w])
        df_test[cn_l] = df_test['LTeamID'].map(lt_mean[cn_l])
   
    df_test = df_test.fillna(0)


elif imputation == 3: # median
    df_test['DayNum'] = df_train['DayNum'].median()
    df_test['NumOT']  = df_train['NumOT'].median()
    
    for i in range(len(w_features)):
        cn_w = w_features[i]
        cn_l = l_features[i]

        df_test[cn_w] = df_test['WTeamID'].map(wt_median[cn_w])
        df_test[cn_l] = df_test['LTeamID'].map(lt_median[cn_l])
   
    df_test = df_test.fillna(0)

# Final df_train
df_train_final = df_train.loc[df_train['WTeamID'].isin(ar_tcr_final_teams)]
df_train_final_indexes = df_train_final.index.values
df_train = df_train.assign(Final_WTeam=0)
df_train_final = df_train_final.assign(Final_WTeam=0)
df_train_final.loc[df_train_final_indexes, 'Final_WTeam'] = df_train_final['WTeamID'].map(df_final_teams_count.set_index('TeamID')['Count'])
df_train.update(df_train_final)

df_train_final = df_train.loc[df_train['LTeamID'].isin(ar_tcr_final_teams)]
df_train_final_indexes = df_train_final.index.values
df_train = df_train.assign(Final_LTeam= 0)
df_train_final = df_train_final.assign(Final_LTeam=0)
df_train_final.loc[df_train_final_indexes, 'Final_LTeam'] = df_train_final['LTeamID'].map(df_final_teams_count.set_index('TeamID')['Count'])
df_train.update(df_train_final)

# Semi final df_train
df_train_final = df_train.loc[df_train['WTeamID'].isin(ar_semi_final_teams)]
df_train_final_indexes = df_train_final.index.values
df_train = df_train.assign(Semi_Final_WTeam=0)
df_train_final = df_train_final.assign(Semi_Final_WTeam=0)
df_train_final.loc[df_train_final_indexes, 'Semi_Final_WTeam'] = df_train_final['WTeamID'].map(df_semi_final_teams_count.set_index('TeamID')['Count'])
df_train.update(df_train_final)

df_train_final = df_train.loc[df_train['LTeamID'].isin(ar_semi_final_teams)]
df_train_final_indexes = df_train_final.index.values
df_train = df_train.assign(Semi_Final_LTeam=0)
df_train_final = df_train_final.assign(Semi_Final_LTeam=0)
df_train_final.loc[df_train_final_indexes, 'Semi_Final_LTeam'] = df_train_final['LTeamID'].map(df_semi_final_teams_count.set_index('TeamID')['Count'])
df_train.update(df_train_final)

'''
df_train['WScore_mean'] = df_train['WTeamID'].map(wt_mean['WScore'])
df_train['LScore_mean'] = df_train['LTeamID'].map(wt_mean['LScore'])
df_train['WScore_median'] = df_train['WTeamID'].map(wt_median['WScore'])
df_train['LScore_median'] = df_train['LTeamID'].map(wt_median['LScore'])
df_train['WScore_sum'] = df_train['WTeamID'].map(wt_sum['WScore'])
df_train['LScore_sum'] = df_train['LTeamID'].map(wt_sum['LScore'])
'''

df_train = df_train.assign(WFGA_mean=0)
df_train = df_train.assign(LFGA_mean=0)
df_train = df_train.assign(WFGA_median=0)
df_train = df_train.assign(LFGA_median=0)
df_train = df_train.assign(WFGA_min=0)
df_train = df_train.assign(LFGA_min=0)

df_train['WFGA_mean'] = df_train['WTeamID'].map(wt_mean['WFGA'])
df_train['LFGA_mean'] = df_train['LTeamID'].map(lt_mean['LFGA'])

df_train['WFGA_median'] = df_train['WTeamID'].map(wt_median['WFGA'])
df_train['LFGA_median'] = df_train['LTeamID'].map(lt_median['LFGA'])

df_train['WFGA_min'] = df_train['WTeamID'].map(wt_min['WFGA'])
df_train['LFGA_min'] = df_train['LTeamID'].map(lt_min['LFGA'])

'''
df_train['WAst_mean'] = df_train['WTeamID'].map(wt_mean['WAst'])
df_train['LAst_mean'] = df_train['LTeamID'].map(lt_mean['LAst'])

df_train['WBlk_mean'] = df_train['WTeamID'].map(wt_mean['WBlk'])
df_train['LBlk_mean'] = df_train['LTeamID'].map(lt_mean['LBlk'])
'''

#df_train['WFGA_max'] = df_train['WTeamID'].map(wt_max['WFGA'])
#df_train['LFGA_max'] = df_train['LTeamID'].map(lt_max['LFGA'])

df_train['WScore_mean'] = df_train.loc[:,['Season','WTeamID']].apply(lambda row: set_aggregation(row, wt_se_agg, 'WTeamID', 'WTeamID', 'mean'), axis=1)
df_train['LScore_mean'] = df_train.loc[:,['Season','LTeamID']].apply(lambda row: set_aggregation(row, lt_se_agg, 'LTeamID', 'LTeamID', 'mean'), axis=1)
df_train['WScore_median'] = df_train.loc[:,['Season','WTeamID']].apply(lambda row: set_aggregation(row, wt_se_agg, 'WTeamID', 'WTeamID', 'median'), axis=1)
df_train['LScore_median'] = df_train.loc[:,['Season','LTeamID']].apply(lambda row: set_aggregation(row, lt_se_agg, 'LTeamID', 'LTeamID', 'median'), axis=1)
df_train['WScore_sum'] = df_train.loc[:,['Season','WTeamID']].apply(lambda row: set_aggregation(row, wt_se_agg, 'WTeamID', 'WTeamID', 'sum'), axis=1)
df_train['LScore_sum'] = df_train.loc[:,['Season','LTeamID']].apply(lambda row: set_aggregation(row, lt_se_agg, 'LTeamID', 'LTeamID', 'sum'), axis=1)

# Counts
df_train['WTeam_W_count'] = OTHER_NAN
df_train['LTeam_L_count'] = OTHER_NAN

count_wt_win = df_train['WTeamID'].map(wt_count['Count'])
count_lt_lose = df_train['LTeamID'].map(lt_count['Count'])
count_wt_lose = df_train['WTeamID'].apply(lambda row: get_value_for_count(row, 'LTeamID', lt_count))
count_lt_win = df_train['LTeamID'].apply(lambda row: get_value_for_count(row, 'WTeamID', wt_count))

df_train['WTeam_W_count'] = count_wt_win
df_train['LTeam_L_count'] = count_lt_lose

df_train['Diff_WTeam'] = count_wt_win - count_wt_lose
df_train['Diff_LTeam'] = count_lt_win - count_lt_lose

df_train['WTeam_PerCent'] = count_wt_win / (count_wt_win + count_wt_lose)
df_train['LTeam_PerCent'] = count_lt_win / (count_lt_win + count_lt_lose)

df_train['WTeam_W_count'] = df_train['WTeam_W_count'].fillna(OTHER_NAN)
df_train['LTeam_L_count'] = df_train['LTeam_L_count'].fillna(OTHER_NAN)


# Preprocess test 

# Test final
df_test_final = df_test.loc[df_test['WTeamID'].isin(ar_tcr_final_teams)]
df_train_final_indexes = df_test_final.index.values
df_test = df_test.assign(Final_WTeam=0)
df_test_final = df_test_final.assign(Final_WTeam=0)
df_test_final.loc[df_train_final_indexes, 'Final_WTeam'] = df_test_final['WTeamID'].map(df_final_teams_count.set_index('TeamID')['Count'])
df_test.update(df_test_final)

df_test_final = df_test[df_test['LTeamID'].isin(ar_tcr_final_teams)]
df_train_final_indexes = df_test_final.index.values
df_test = df_test.assign(Final_LTeam=0)
df_test_final = df_test_final.assign(Final_LTeam=0)
df_test_final.loc[df_train_final_indexes, 'Final_LTeam'] = df_test_final['LTeamID'].map(df_final_teams_count.set_index('TeamID')['Count'])
df_test.update(df_test_final)



# Test semi final
df_test_semi_final = df_test.loc[df_test['WTeamID'].isin(ar_semi_final_teams)]

df_test_final_indexes = df_test_semi_final.index.values
df_test['Semi_Final_WTeam'] = 0
df_test_semi_final['Semi_Final_WTeam'] = 0
df_test_semi_final.loc[df_test_final_indexes, 'Semi_Final_WTeam'] = df_test_semi_final['WTeamID'].map(df_semi_final_teams_count.set_index('TeamID')['Count'])
df_test.update(df_test_semi_final)

df_test_semi_final = df_test[df_test['LTeamID'].isin(ar_semi_final_teams)]
df_test_final_indexes = df_test_semi_final.index.values
df_test['Semi_Final_LTeam'] = 0
df_test_semi_final['Semi_Final_LTeam'] = 0
df_test_semi_final.loc[df_test_final_indexes, 'Semi_Final_LTeam'] = df_test_semi_final['LTeamID'].map(df_semi_final_teams_count.set_index('TeamID')['Count'])
df_test.update(df_test_semi_final)


'''
df_test['WScore_mean'] = df_test['WTeamID'].map(wt_mean['WScore'])
df_test['LScore_mean'] = df_test['LTeamID'].map(wt_mean['LScore'])
df_test['WScore_median'] = df_test['WTeamID'].map(wt_median['WScore'])
df_test['LScore_median'] = df_test['LTeamID'].map(wt_median['LScore'])
df_test['WScore_sum'] = df_test['WTeamID'].map(wt_sum['WScore'])
df_test['LScore_sum'] = df_test['LTeamID'].map(wt_sum['LScore'])
'''

df_test['WFGA_mean'] = df_test['WTeamID'].map(wt_mean['WFGA'])
df_test['LFGA_mean'] = df_test['LTeamID'].map(lt_mean['LFGA'])

df_test['WFGA_median'] = df_test['WTeamID'].map(wt_median['WFGA'])
df_test['LFGA_median'] = df_test['LTeamID'].map(lt_median['LFGA'])

df_test['WFGA_min'] = df_test['WTeamID'].map(wt_min['WFGA'])
df_test['LFGA_min'] = df_test['LTeamID'].map(lt_min['LFGA'])

print("Here 8")

'''
df_test['WAst_mean'] = df_test['WTeamID'].map(wt_mean['WAst'])
df_test['LAst_mean'] = df_test['LTeamID'].map(lt_mean['LAst'])

df_test['WBlk_mean'] = df_test['WTeamID'].map(wt_mean['WBlk'])
df_test['LBlk_mean'] = df_test['LTeamID'].map(lt_mean['LBlk'])
'''

#df_test['WFGA_max'] = df_test['WTeamID'].map(wt_max['WFGA'])
#df_test['LFGA_max'] = df_test['LTeamID'].map(lt_max['LFGA'])

df_test['WScore_mean'] = df_test.loc[:,['Season','WTeamID']].apply(lambda row: set_aggregation(row, wt_se_agg, 'WTeamID', 'WTeamID', 'mean'), axis=1)
df_test['LScore_mean'] = df_test.loc[:,['Season','LTeamID']].apply(lambda row: set_aggregation(row, lt_se_agg, 'LTeamID', 'LTeamID', 'mean'), axis=1)
df_test['WScore_median'] = df_test.loc[:,['Season','WTeamID']].apply(lambda row: set_aggregation(row, wt_se_agg, 'WTeamID', 'WTeamID', 'median'), axis=1)
df_test['LScore_median'] = df_test.loc[:,['Season','LTeamID']].apply(lambda row: set_aggregation(row, lt_se_agg, 'LTeamID', 'LTeamID', 'median'), axis=1)
df_test['WScore_sum'] = df_test.loc[:,['Season','WTeamID']].apply(lambda row: set_aggregation(row, wt_se_agg, 'WTeamID', 'WTeamID', 'sum'), axis=1)
df_test['LScore_sum'] = df_test.loc[:,['Season','LTeamID']].apply(lambda row: set_aggregation(row, lt_se_agg, 'LTeamID', 'LTeamID', 'sum'), axis=1)

print("Here 9")

# Counts
count_wt_win = df_test['WTeamID'].map(wt_count['Count'])
count_lt_lose = df_test['LTeamID'].map(lt_count['Count'])
count_wt_lose = df_test['WTeamID'].apply(lambda row: get_value_for_count(row, 'LTeamID', lt_count))
count_lt_win = df_test['LTeamID'].apply(lambda row: get_value_for_count(row, 'WTeamID', wt_count))

df_test = df_test.assign(WTeam_W_count=OTHER_NAN)
df_test = df_test.assign(LTeam_L_count=OTHER_NAN)

df_test.loc[:, 'WTeam_W_count'] = count_wt_win
df_test.loc[:, 'LTeam_L_count'] = count_lt_lose

df_test['WTeam_W_count'] = df_test['WTeam_W_count'].fillna(OTHER_NAN)
df_test['LTeam_L_count'] = df_test['LTeam_L_count'].fillna(OTHER_NAN)

df_test['Diff_WTeam'] = count_wt_win - count_wt_lose
df_test['Diff_LTeam'] = count_lt_win - count_lt_lose

df_test['Diff_WTeam'] = df_test['Diff_WTeam'].fillna(OTHER_NAN)
df_test['Diff_LTeam'] = df_test['Diff_LTeam'].fillna(OTHER_NAN)

df_test['WTeam_PerCent'] = count_wt_win / (count_wt_win + count_wt_lose)
df_test['LTeam_PerCent'] = count_lt_win / (count_lt_win + count_lt_lose)

df_test['WTeam_PerCent'] = df_test['WTeam_PerCent'].fillna(OTHER_NAN)
df_test['LTeam_PerCent'] = df_test['LTeam_PerCent'].fillna(OTHER_NAN)

features = df_train.columns

category_features_names = ['Season', 'DayNum', 'WLoc', 'WTeamID', 'LTeamID']

df_train = df_train.fillna(OTHER_NAN)
df_test = df_test.fillna(OTHER_NAN)

print("Here 10")

#df_train[category_features_names] = df_train[category_features_names].astype('int64').astype('category')
#df_test[category_features_names] = df_test[category_features_names].astype('int64').astype('category')

x1 = df_train.shape[0]
df_train_inverse = df_train.copy()

for i in range(len(W_FEATURES)):
    v_w = W_FEATURES[i]
    v_l = L_FEATURES[i]
    df_train_inverse[v_w] = df_train[v_l]
    df_train_inverse[v_l] = df_train[v_w]

df_train_inverse['WLoc'] = df_train_inverse['WLoc'].apply(set_WLoc)    
df_train = df_train.append(df_train_inverse, ignore_index=True)

train_features = [cn for cn in df_train.columns if cn not in ['WScore', 'LScore', "NumOT"]]
Y = df_train['WScore'] - df_train['LScore']

mms = MinMaxScaler()
mms.fit(Y.values.reshape(-1,1))
yyy = mms.transform(Y.values.reshape(-1,1))
Y = pd.DataFrame(data=yyy, columns=["Y"])

X_train = df_train[train_features]
X_test = df_test[train_features]

X_train[category_features_names] = X_train[category_features_names].astype('int64').astype('category')
X_test[category_features_names] = X_test[category_features_names].astype('int64').astype('category')

final_encoding = 1
cat_features = []

if final_encoding==0: # all data encoded with TE, cat_features empty
    X_train = X_train.applymap(lambda x: str(x))
    X_test = X_test.applymap(lambda x: str(x))
    te = ce.TargetEncoder(smoothing=0.2)
    te.fit(X_train, Y)
    X_train = te.transform(X_train, Y)
    X_test = te.transform(X_test)
    cat_features = []
elif final_encoding==1: # TE only on category features, cat_features = empty
    te = ce.TargetEncoder(cols=category_features_names, smoothing=0.2)
    te.fit(X_train, Y)
    X_train = te.transform(X_train, Y)
    X_test = te.transform(X_test)
    cat_features = []
    # cat_features = category_features_names
elif final_encoding==2:
    non_cat = [cn for cn in X_train.columns if cn not in category_features_names]
    
    X_train_numeric = X_train[non_cat].applymap(lambda x: str(x))
    X_test_numeric = X_test[non_cat].applymap(lambda x: str(x))

    te = ce.TargetEncoder(smoothing=0.2)
    te.fit(X_train_numeric, Y)
    X_train_numeric = te.transform(X_train_numeric, Y)
    X_test_numeric = te.transform(X_test_numeric)
    
    X_train[non_cat] = X_train_numeric
    X_test.update(X_test_numeric)
    cat_features = category_features_names
elif final_encoding==3: # all data without cat features
    X_train = X_train.drop(['Season', 'DayNum', 'WTeamID', 'LTeamID'], axis=1)
    X_test = X_test.drop(['Season', 'DayNum', 'WTeamID', 'LTeamID'], axis=1)
    #cat_features = ['WLoc']
else:
    cat_features = category_features_names

X = X_train
X_testset= X_test


print('Max train :')
print(X.max().max())

print('Max test :')
print(X_testset.max().max())

'''
print("Has 8888 test ")
print(X_testset[X_testset.eq(8888.0).any(1)])

print("Has 8888 train ")
print(X[X.eq(8888.0).any(1)])
'''

names = [
         #"Ridge",
         #"RidgeCV",
         #"LinearSVM",
         "XGB_Regressor", 
         "XGBOOST",  
         "GBC_Regressor",
         "HGBC_Regressor",
         "ETC_Regressor",
         #"LDA",
         #"QDA",
         #"GaussianProcess",
         "DecisionTree",
         "RandomForest_Regressor",
         "AdaBoost_Regressor",
         "NaiveBayes",
         "LogisticRegression",
         #"RBF SVM",
         "CatBoost_Regressor",
         "LGB",   
         "Huber_Regressor",
         "Theil_Regressor"
         #"NeuralNet"
         #"NearestNeighbors"
    ]

classifiers = [
        #RidgeClassifier(),
        #RidgeClassifierCV(),
        #SVC(kernel="linear", C=0.025),
        XGBRegressor(),
        xgb,
        GradientBoostingRegressor(),
        HistGradientBoostingRegressor(),
        ExtraTreesRegressor(),
        #LinearDiscriminantAnalysis(),
        #QuadraticDiscriminantAnalysis(),
        #GaussianProcessClassifier(1.0 * RBF(1.0)),
        DecisionTreeRegressor(max_depth=5),
        RandomForestRegressor(max_depth=5, n_estimators=500),
        AdaBoostRegressor(),
        GaussianNB(),
        LogisticRegression(max_iter=10000),
        #SVC(gamma=2, C=1),
        CatBoostRegressor(**PARAMS_CATBOOST_REGRESSOR),
        lgb,
        HuberRegressor(),
        TheilSenRegressor()
        #MLPClassifier(alpha=0.1, max_iter=50, batch_size=50, tol=0.0001, verbose=False)
        #KNeighborsClassifier()
    ]

names = [
         #"LogisticRegression",
         #"RBF SVM",
         "CatBoost_Regressor",
         "LGB",   
         "Huber_Regressor",
         "Theil_Regressor"
         #"NeuralNet"
         #"NearestNeighbors"
    ]

classifiers = [
        #LogisticRegression(max_iter=10000),
        #SVC(gamma=2, C=1),
        CatBoostRegressor(**PARAMS_CATBOOST_REGRESSOR),
        lgb,
        HuberRegressor(),
        TheilSenRegressor()
        #MLPClassifier(alpha=0.1, max_iter=50, batch_size=50, tol=0.0001, verbose=False)
        #KNeighborsClassifier()
    ]

#names = ["XGBOOST", "LGB"]
#classifiers = [xgb, lgb]

def vectorize_switch_zero_one(x):
    if x == 1:
        return 0
    else:
        return 1

function_vectorize_switch_zero_one = np.vectorize(vectorize_switch_zero_one)
function_vectorize_pred_result = np.vectorize(vectorize_pred_result)

SPLITS = 3
kf = KFold(n_splits=SPLITS, shuffle=True, random_state=SEED)

format_data = 'df'

for name, clf in zip(names, classifiers):
    print("Classifier "+name)
        
    test_preds = np.zeros((X_testset.shape[0],1))
    test_score = 0
    train_score = 0
    p_test_score = 0
    count = 0
    
    if name in ["CatBoost_Classifier", "CatBoost_Regressor"]:
        pass
    else:
        X = X.astype('float64')
        X_testset = X_testset.astype('float64')
    
    """
    clf.fit(X, Y.values.ravel()) 
    y_pred_proba = clf.predict(X_testset) 
    y_pred_proba = y_pred_proba / float(10)
     
    y_pred_proba = function_vectorize_pred_result(y_pred_proba) 
    print_submission_into_file(y_pred_proba, df_test_id, "_"+str(name))
    """
    oof = np.zeros((X.shape[0],1))
    
    for train_index, test_index in kf.split(X, Y):
        count = count+1
        print("Split "+str(count)+" ... "+name)
     
        if format_data == 'np':
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = Y[train_index], Y[test_index]
        elif format_data == 'df':
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]
        
        if name in ["CatBoost_Classifier", "CatBoost_Regressor"]:
            train_dataset = Pool(data=X_train, label=y_train, cat_features=cat_features)
            eval_dataset = Pool(data=X_test, label=y_test, cat_features=cat_features)
            clf.fit(
                train_dataset, 
                use_best_model=True, 
                eval_set=[eval_dataset], 
                plot=True
            ) # Get predicted classes
     
            print("Count of trees in model = {}".format(clf.tree_count_))
        
        elif name == "LGB":
            train_data = lgb.Dataset(X_train.to_numpy(), label=y_train.values.flatten())
            validation_data = lgb.Dataset(X_test.to_numpy(), label=y_test.values.flatten())
            prediction_data = lgb.Dataset(X_testset.to_numpy())
            
            #clf = lgb.train(PARAM_LGB, train_data, valid_sets=[validation_data])
            clf = lgb.train(lgb_params, train_data, valid_sets=[validation_data])
            
        elif name == "XGBOOST":
            train_set = xgb.DMatrix(X_train, y_train)
            val_set = xgb.DMatrix(X_test, y_test)
            prediction_data = xgb.DMatrix(X_testset)
            clf = xgb.train(xgb_params, train_set, num_boost_round=4000, evals=[(train_set, 'train'), (val_set, 'val')], verbose_eval=100)
        else:
            #clf.fit(X_train, y_train.values.ravel())
            clf.fit(X_train, y_train.values)
        
        # save the model to disk
        #filename = 'model_ALL_'+str(SPLITS)+'_splits_'+name+'_'+str(count)+'.sav'
        #pickle.dump(clf, open(filename, 'wb'))
           
        if name == "XGBOOST":    
            y_train_predict = clf.predict(train_set)
            y_test_predict = clf.predict(val_set)
            y_pred_proba = clf.predict(prediction_data)
        else:
            y_train_predict = clf.predict(X_train)
            y_test_predict = clf.predict(X_test)
            y_pred_proba = clf.predict(X_testset)    
        
        #print("prediction shape", y_pred_proba.shape, y_train.shape, y_test.shape)
        
        if name in ["LGB", "XGBOOST"]:
            score_test = mean_squared_error(y_test, y_test_predict)
            score_train = mean_squared_error(y_train, y_train_predict)
        else:
            score_test = clf.score(X_test, y_test)
            score_train = clf.score(X_train, y_train)
        
            #y_test_predict = y_test_predict.reshape(-1, 1)
            #y01 = y_test.to_numpy().reshape((y_test.shape[0], 1))
            #p = log_loss(y01, y_test_predict)
            '''
            y_train_predict = y_train_predict.reshape(-1, 1)
            y01 = y_train.to_numpy().reshape((y_train.shape[0], 1))
            pp = log_loss(y01, y_train_predict)
            '''
            #oof[test_index] = y_test_predict

            print("Score Test : "+str(score_test))
            #print("Score Train : "+str(score_train))
            #print("Score Test : "+str(p))
            #print("Score Train : "+str(pp))

            test_score += score_test
            #p_test_score += p
            # train_score += score_train
            #test_preds += y_pred_proba/float(SPLITS)
        test_preds += y_pred_proba.reshape(-1,1)
        
        #print_submission_into_file(y_pred_proba, df_test_id, "_ALL_"+str(name)+'_'+str(SPLITS)+'_splits_'+str(count))
    
    test_preds /= float(SPLITS)
    test_preds = test_preds / float(10)
     
    test_preds = function_vectorize_pred_result(test_preds) 
    
    stest = test_score / float(SPLITS)
    #s_rmselog = p_test_score / float(SPLITS)
    print("Score Test : "+str(stest))
    #print("Score Test RMSE LOG : "+str(s_rmselog))
    '''
    strain = train_score / float(SPLITS)
    print("Score Train : "+str(strain))
    '''
    
    test_preds = test_preds.flatten().flatten()
    print_submission_into_file(test_preds.ravel(), df_test_id, "_S2_"+str(name))
    
    if name == "XGBOOST":
        print_submission(test_preds.ravel(), df_test_id)

    #score_test = accuracy_score(Y.to_numpy(), oof.flatten())
    #print("Score Test FINAL : "+str(score_test))
    
    
    
print("OK")